In [9]:
# Azure Machine Learning SDK core
from azureml.core import Workspace
from azureml.core.model import Model

# Scikit-learn and others
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import pickle

### Load and connect to workspace

In [10]:
ws = Workspace.from_config("Azure_machine_learning/config.json")

### Registering model onto Azure

In [11]:
model = Model.register(ws, model_name="data_salaries_random_forest_regression", model_path="model.pkl")

Registering model data_salaries_random_forest_regression


### Test and share endpoint for marking

In [12]:
import urllib.request
import json
import os
import ssl

def allowSelfSignedHttps(allowed):
    # bypass the server certificate verification on client side
    if allowed and not os.environ.get('PYTHONHTTPSVERIFY', '') and getattr(ssl, '_create_unverified_context', None):
        ssl._create_default_https_context = ssl._create_unverified_context

allowSelfSignedHttps(True) # this line is needed if you use self-signed certificate in your scoring service.

# Request data goes here
# The example below assumes JSON formatting which may be updated
# depending on the format your endpoint expects.
# More information can be found here:
# https://docs.microsoft.com/azure/machine-learning/how-to-deploy-advanced-entry-script
data = {
    "data": [[500], [10000]]
}

body = str.encode(json.dumps(data))

url = 'https://msa2023-phase2-azure-przto.australiaeast.inference.ml.azure.com/score'
# Replace this with the primary/secondary key or AMLToken for the endpoint
api_key = 'uk8j9RkV2pEsssu2lBxK2yfwz719CyOz'
if not api_key:
    raise Exception("A key should be provided to invoke the endpoint")

# The azureml-model-deployment header will force the request to go to a specific deployment.
# Remove this header to have the request observe the endpoint traffic rules
headers = {'Content-Type':'application/json', 'Authorization':('Bearer '+ api_key), 'azureml-model-deployment': 'data-salaries-random-forest-re-1' }

req = urllib.request.Request(url, body, headers)

try:
    response = urllib.request.urlopen(req)

    result = response.read()
    print(result)
except urllib.error.HTTPError as error:
    print("The request failed with status code: " + str(error.code))

    # Print the headers - they include the requert ID and the timestamp, which are useful for debugging the failure
    print(error.info())
    print(error.read().decode("utf8", 'ignore'))


The request failed with status code: 424
server: azureml-frontdoor
date: Sun, 06 Aug 2023 10:55:28 GMT
content-type: application/json
content-length: 92
x-ms-run-function-failed: True
x-ms-server-version: azmlinfsrv/0.8.4
x-ms-request-id: 148b6fd3-78dd-4170-bf72-620eb10b7c60
x-request-id: 148b6fd3-78dd-4170-bf72-620eb10b7c60
ms-azureml-model-error-reason: model_error
ms-azureml-model-error-statuscode: 500
azureml-model-deployment: data-salaries-random-forest-re-1
connection: close


{"message": "An unexpected error occurred in scoring script. Check the logs for more info."}
